In [60]:
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import time
import pickle
import pprint
import json

In [3]:
location_raw_data = {}

# Load raw location data - used the Nominatim Geolocator

In [124]:

def get_raw_data():
    """Loop through file, obtain reverse lookup location metadata, 
        store in dict and write to file."""
    
    geolocator = Nominatim(user_agent="test_API")

    df = pd.read_csv('lat_lon.csv')
    
    #Remove duplicates to reduce API calls
    df = df.drop_duplicates()

    for index, row in df.iterrows():
        lat_lon = str(row['lat_four']) + ", "+ str(row['lng_four'])
        location = geolocator.reverse(lat_lon)
        location_raw_data[lat_lon] = location.raw
        print(lat_lon)
        time.sleep(1)

    pickle.dump(location_raw_data, open("loc_raw_data.p", "wb"))
    print (location_raw_data)

    

def load_pickle_data():
    raw_data = pickle.load(open ("loc_raw_data.p", "rb"))
    return raw_data




# ETL / DATA CLEANING
The goal here is to have every lat/lon coord placed in a state district.

Next we will aim to place each coord in at least ONE of the following:
- City
- Town
- Village
- Hamlet


############### ETL STEPS - Uncomment as necessary  ##################

def map_address_to_lat_long(row):
    """Looks for the address element of each lat lon element
       Return a series containing the CITY, TOWN, VILLAGE and STATE_DISTRICT
       Elements of the address"""
    
    lat_lon = str(row['lat_four']) + ", "+ str(row['lng_four'])

    city = raw_location_data[lat_lon]['address'].get('city', None)
    town = raw_location_data[lat_lon]['address'].get('town', None)
    village = raw_location_data[lat_lon]['address'].get('village', None)
    hamlet = raw_location_data[lat_lon]['address'].get('hamlet', None)
    state_district = raw_location_data[lat_lon]['address'].get('state_district', None)
    
    return pd.Series([city, town, village, hamlet, state_district], 
                     index=['city', 'town', 'village', 'hamlet', 'state_district'])

#Get raw location data for each of the lat/long coords
#get_raw_data()

#Load raw data
raw_location_data = load_pickle_data()

df = pd.read_csv('lat_lon.csv')

#We are interested in the City, Town, State_District each of the trips takes place in.
df[['city','town', 'village', 'hamlet', 'state_district']] = df.apply(map_address_to_lat_long, axis=1)

#Load this to CSV, check results
#df.to_csv('lat_lon_full.csv')


# Missing State Districts

In [127]:
#I noticed there was one element that didn't have a state_district.
df_not_mapped = df[df['state_district'].isnull()]

# There was no clues in the raw data...
print(location_raw_data["53.1794, -2.9438"])

#So I just manually looked up the site and fixed it...

#Manually ammended this element
df.loc[df_not_mapped.index[0], 'state_district'] = "North West"


{'place_id': 70147557, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 6008815, 'lat': '53.1791206458678', 'lon': '-2.94376740910927', 'display_name': 'Bradshaw Avenue, Saltney, Saltney Ferry, Flintshire, Wales, CH4 0AZ, UK', 'address': {'road': 'Bradshaw Avenue', 'suburb': 'Saltney', 'village': 'Saltney Ferry', 'county': 'Flintshire', 'state': 'Wales', 'postcode': 'CH4 0AZ', 'country': 'UK', 'country_code': 'gb'}, 'boundingbox': ['53.179013', '53.1791409', '-2.9446901', '-2.9434533']}
